In [25]:
# pip install soynlp -q

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [6]:
import re
from soynlp.normalizer import *

def preprocessing(text):
    # 문제를 일으킬 수 있는 문자 제거
    bad_chars = {"\u200b": "", "…": " ... ", "\ufeff": ""}
    for bad_char in bad_chars:
        text = text.replace(bad_char, bad_chars[bad_char])
        
    error_chars = {"\u3000": " ", "\u2009": " ", "\u2002": " ", "\xa0":" "}
    for error_char in error_chars:
        text = text.replace(error_char, error_chars[error_char])
    
    # 이메일 제거
    text = re.sub(r"[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+", " ", text).strip()
    
    # "#문자" 형식 어절 제거
    text = re.sub(r"#\S+", "", text).strip()
    
    # "@문자" 형식 어절 제거
    text = re.sub(r"@\w+", "", text).strip()
    
    # URL 제거
    text = re.sub(r"(http|https)?:\/\/\S+\b|www\.(\w+\.)+\S*", " ", text).strip()
    text = re.sub(r"pic\.(\w+\.)+\S*", " ", text).strip()
    
    # 뉴스 저작권 관련 텍스트 제거
    re_patterns = [
        r"\<저작권자(\(c\)|ⓒ|©|\(Copyright\)|(\(c\))|(\(C\))).+?\>",
        r"저작권자\(c\)|ⓒ|©|(Copyright)|(\(c\))|(\(C\))"
    ]
    
    for re_pattern in re_patterns:
        text = re.sub(re_pattern, "", text).strip()
    
    # 뉴스 내 포함된 이미지에 대한 레이블 제거
    text = re.sub(r"\(출처 ?= ?.+\) |\(사진 ?= ?.+\) |\(자료 ?= ?.+\)| \(자료사진\) |사진=.+기자 ", "", text).strip()
    
    # 중복 문자 처리
    text = repeat_normalize(text, num_repeats=2).strip()
    
    # 문제를 일으킬 수 있는 구두점 치환
    punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', }
    for p in punct_mapping:
        text = text.replace(p, punct_mapping[p])
    
    # 연속된 공백 치환
    text = re.sub(r"\s+", " ", text).strip()
    
    # 개행 문자 "\n" 제거
    text = text.replace('\n', '')
    
    return text

In [21]:
import pandas as pd
import os

# pp_df는 전처리 된 데이터
pp_df = pd.DataFrame({'context':[]})

keyword = 'episode_2'
data_dir = f'/Users/yongchanchun/Desktop/MacBook_Pro_Desktop/해커톤/{keyword}'
save_dir = f'/Users/yongchanchun/Desktop/MacBook_Pro_Desktop/해커톤/preprocessed_{keyword}'

# 3. 카테고리별로 폴더 생성하는 함수
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)

createFolder(save_dir)

for text_name in sorted(os.listdir(data_dir)):
    text_dir = os.path.join(data_dir, text_name)
    with open(text_dir, 'r') as f:
        raw_data = f.read()
        if raw_data == '':
            continue
        preprocessed_text = preprocessing(raw_data)
        pp_df = pp_df.append({'context': preprocessed_text}, ignore_index=True)
   
pp_df.head(10)
pp_df.to_csv(os.path.join(save_dir, f'{keyword}.txt'), sep='\t', header=False, index=False)

with open(os.path.join(save_dir, f'{keyword}_merged.txt'), 'w') as f:
    for i, text in enumerate(pp_df['context']):
        if i != 0:
            f.write('\n'+text)
        else:
            f.write(text)